# Quickstart

Tutorial for [NautilusTrader](https://nautilustrader.io/docs/) a high-performance algorithmic trading platform and event driven backtester.

[View source on GitHub](https://github.com/nautechsystems/nautilus_trader/blob/develop/docs/getting_started/quickstart.ipynb).

## Overview

This quickstart tutorial steps through how to get up and running with NautilusTrader backtesting using FX data.
To support this, some pre-loaded test data is available using the standard Nautilus persistence format (Parquet).

## Prerequisites
- Python 3.11+ installed
- [NautilusTrader](https://pypi.org/project/nautilus_trader/) latest release installed (`pip install -U nautilus_trader`)
- [JupyterLab](https://jupyter.org/) or similar installed (`pip install -U jupyterlab`)

## 1. Get sample data

To save time, we have prepared a script to load sample data into the Nautilus format for use with this example. 
First, download and load the data by running the next cell (this should take ~ 1-2 mins):

```python
import runpy
import tempfile
import urllib.request

with tempfile.NamedTemporaryFile("wb", suffix=".py", delete=False) as f:
    f.write(urllib.request.urlopen(
        "https://raw.githubusercontent.com/nautechsystems/nautilus_data/main/nautilus_data/hist_data_to_catalog.py"
    ).read())
runpy.run_path(f.name, run_name="__main__")
```

For further details on how to load data into Nautilus, see [Loading External Data](https://nautilustrader.io/docs/latest/concepts/data#loading-data) guide.

In [1]:
from nautilus_trader.backtest.node import BacktestDataConfig
from nautilus_trader.backtest.node import BacktestEngineConfig
from nautilus_trader.backtest.node import BacktestNode
from nautilus_trader.backtest.node import BacktestRunConfig
from nautilus_trader.backtest.node import BacktestVenueConfig
from nautilus_trader.config import ImportableStrategyConfig
from nautilus_trader.config import LoggingConfig
from nautilus_trader.model import Quantity
from nautilus_trader.model import QuoteTick
from nautilus_trader.persistence.catalog import ParquetDataCatalog

## 2. Set up a Parquet data catalog

If everything worked correctly, you should be able to see a single EUR/USD instrument in the catalog.

In [4]:
# You can also use a relative path such as `ParquetDataCatalog("w`,
# for example if you're running this notebook after the data setup from the docs.
catalog = ParquetDataCatalog("./catalog")
# catalog = ParquetDataCatalog.from_env()
catalog.instruments()

[CurrencyPair(id=EUR/USD.SIM, raw_symbol=EUR/USD, asset_class=FX, instrument_class=SPOT, quote_currency=USD, is_inverse=False, price_precision=5, price_increment=0.00001, size_precision=0, size_increment=1, multiplier=1, lot_size=1000, margin_init=0.03, margin_maint=0.03, maker_fee=0.00002, taker_fee=0.00002, info=None)]

## 3. Write a trading strategy

NautilusTrader includes many indicators built-in, in this example we will use the MACD indicator to 
build a simple trading strategy.

You can read more about [MACD here](https://www.investopedia.com/terms/m/macd.asp), this 
indicator merely serves as an example without any expected alpha. There is also a way of
registering indicators to receive certain data types, however in this example we manually pass the received
`QuoteTick` to the indicator in the `on_quote_tick` method.

In [6]:
from nautilus_trader.core.message import Event
from nautilus_trader.indicators.macd import MovingAverageConvergenceDivergence
from nautilus_trader.model import InstrumentId
from nautilus_trader.model import Position
from nautilus_trader.model.enums import OrderSide
from nautilus_trader.model.enums import PositionSide
from nautilus_trader.model.enums import PriceType
from nautilus_trader.model.events import PositionOpened
from nautilus_trader.trading.strategy import Strategy
from nautilus_trader.trading.strategy import StrategyConfig


class MACDConfig(StrategyConfig):
    instrument_id: InstrumentId
    fast_period: int = 12
    slow_period: int = 26
    trade_size: int = 1_000_000
    entry_threshold: float = 0.00010


class MACDStrategy(Strategy):
    def __init__(self, config: MACDConfig):
        super().__init__(config=config)
        # Our "trading signal"
        self.macd = MovingAverageConvergenceDivergence(
            fast_period=config.fast_period, slow_period=config.slow_period, price_type=PriceType.MID
        )

        self.trade_size = Quantity.from_int(config.trade_size)

        # Convenience
        self.position: Position | None = None

    def on_start(self):
        self.subscribe_quote_ticks(instrument_id=self.config.instrument_id)

    def on_stop(self):
        self.close_all_positions(self.config.instrument_id)
        self.unsubscribe_quote_ticks(instrument_id=self.config.instrument_id)

    def on_quote_tick(self, tick: QuoteTick):
        # You can register indicators to receive quote tick updates automatically,
        # here we manually update the indicator to demonstrate the flexibility available.
        self.macd.handle_quote_tick(tick)

        if not self.macd.initialized:
            return  # Wait for indicator to warm up

        # self._log.info(f"{self.macd.value=}:%5d")
        self.check_for_entry()
        self.check_for_exit()

    def on_event(self, event: Event):
        if isinstance(event, PositionOpened):
            self.position = self.cache.position(event.position_id)

    def check_for_entry(self):
        # If MACD line is above our entry threshold, we should be LONG
        if self.macd.value > self.config.entry_threshold:
            if self.position and self.position.side == PositionSide.LONG:
                return  # Already LONG

            order = self.order_factory.market(
                instrument_id=self.config.instrument_id,
                order_side=OrderSide.BUY,
                quantity=self.trade_size,
            )
            self.submit_order(order)
        # If MACD line is below our entry threshold, we should be SHORT
        elif self.macd.value < -self.config.entry_threshold:
            if self.position and self.position.side == PositionSide.SHORT:
                return  # Already SHORT

            order = self.order_factory.market(
                instrument_id=self.config.instrument_id,
                order_side=OrderSide.SELL,
                quantity=self.trade_size,
            )
            self.submit_order(order)

    def check_for_exit(self):
        # If MACD line is above zero then exit if we are SHORT
        if self.macd.value >= 0.0:
            if self.position and self.position.side == PositionSide.SHORT:
                self.close_position(self.position)
        # If MACD line is below zero then exit if we are LONG
        else:
            if self.position and self.position.side == PositionSide.LONG:
                self.close_position(self.position)

    def on_dispose(self):
        pass  # Do nothing else

## Configuring backtests

Now that we have a trading strategy and data, we can begin to configure a backtest run. Nautilus uses a `BacktestNode` 
to orchestrate backtest runs, which requires some setup. This may seem a little complex at first, 
however this is necessary for the capabilities that Nautilus strives for.

To configure a `BacktestNode`, we first need to create an instance of a `BacktestRunConfig`, configuring the 
following (minimal) aspects of the backtest:

- `engine`: The engine for the backtest representing our core system, which will also contain our strategies
- `venues`: The simulated venues (exchanges or brokers) available in the backtest
- `data`: The input data we would like to perform the backtest on

There are many more configurable features which will be described later in the docs, for now this will get us up and running.

## 4. Configure venue

First, we create a venue configuration. For this example we will create a simulated FX ECN. 
A venue needs a name which acts as an ID (in this case `SIM`), as well as some basic configuration, e.g. 
the account type (`CASH` vs `MARGIN`), an optional base currency, and starting balance(s).

:::note
FX trading is typically done on margin with Non-Deliverable Forward, Swap or CFD type instruments.
:::

In [7]:
venue = BacktestVenueConfig(
    name="SIM",
    oms_type="NETTING",
    account_type="MARGIN",
    base_currency="USD",
    starting_balances=["1_000_000 USD"]
)

## 5. Configure data

We need to know about the instruments that we would like to load data for, we can use the `ParquetDataCatalog` for this.

In [8]:
instruments = catalog.instruments()
instruments

[CurrencyPair(id=EUR/USD.SIM, raw_symbol=EUR/USD, asset_class=FX, instrument_class=SPOT, quote_currency=USD, is_inverse=False, price_precision=5, price_increment=0.00001, size_precision=0, size_increment=1, multiplier=1, lot_size=1000, margin_init=0.03, margin_maint=0.03, maker_fee=0.00002, taker_fee=0.00002, info=None)]

Next, we need to configure the data for the backtest. Nautilus is built to be very flexible when it 
comes to loading data for backtests, however this also means some configuration is required.

For each tick type (and instrument), we add a `BacktestDataConfig`. In this instance we are simply 
adding the `QuoteTick`(s) for our EUR/USD instrument:

In [9]:
from nautilus_trader.model import QuoteTick


data = BacktestDataConfig(
    catalog_path=str(catalog.path),
    data_cls=QuoteTick,
    instrument_id=instruments[0].id,
    end_time="2020-01-10",
)

## 6. Configure engine

Then, we need a `BacktestEngineConfig` which represents the configuration of our core trading system.
Here we need to pass our trading strategies, we can also adjust the log level 
and configure many other components (however, it's also fine to use the defaults):

Strategies are added via the `ImportableStrategyConfig`, which enables importing strategies from arbitrary files or 
user packages. In this instance, our `MACDStrategy` is defined in the current module, which python refers to as `__main__`.

In [10]:
# NautilusTrader currently exceeds the rate limit for Jupyter notebook logging (stdout output),
# this is why the `log_level` is set to "ERROR". If you lower this level to see
# more logging then the notebook will hang during cell execution. A fix is currently
# being investigated which involves either raising the configured rate limits for
# Jupyter, or throttling the log flushing from Nautilus.
# https://github.com/jupyterlab/jupyterlab/issues/12845
# https://github.com/deshaw/jupyterlab-limit-output
engine = BacktestEngineConfig(
    strategies=[
        ImportableStrategyConfig(
            strategy_path="__main__:MACDStrategy",
            config_path="__main__:MACDConfig",
            config={
              "instrument_id": instruments[0].id,
              "fast_period": 12,
              "slow_period": 26,
            },
        )
    ],
    logging=LoggingConfig(log_level="ERROR"),
)

## 7. Run backtest

We can now pass our various config pieces to the `BacktestRunConfig`. This object now contains the 
full configuration for our backtest.

In [11]:
config = BacktestRunConfig(
    engine=engine,
    venues=[venue],
    data=[data],
)

The `BacktestNode` class will orchestrate the backtest run. The reason for this separation between 
configuration and execution is the `BacktestNode`, which enables running multiple configurations (different 
parameters or batches of data). We are now ready to run some backtests.

In [12]:
from nautilus_trader.backtest.results import BacktestResult


node = BacktestNode(configs=[config])

 # Runs one or many configs synchronously
results: list[BacktestResult] = node.run()

## 8. Analyze results

Now that the run is complete, we can also directly query for the `BacktestEngine`(s) used internally by the `BacktestNode`
by using the run configs ID. 

The engine(s) can provide additional reports and information.

In [13]:
from nautilus_trader.backtest.engine import BacktestEngine
from nautilus_trader.model import Venue


engine: BacktestEngine = node.get_engine(config.id)

engine.trader.generate_order_fills_report()

,trader_id,strategy_id,instrument_id,venue_order_id,position_id,account_id,last_trade_id,type,side,quantity,...,order_list_id,linked_order_ids,parent_order_id,exec_algorithm_id,exec_algorithm_params,exec_spawn_id,tags,init_id,ts_init,ts_last
client_order_id,,,,,,,,,,,,,,,,,,,,,
O-20200103-100000-001-000-1,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-001,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-001,MARKET,BUY,1000000,...,None,None,None,None,None,None,None,794d2361-7a60-46af-8e12-fe86baf55fc1,2020-01-03 10:00:00.242000+00:00,2020-01-03 10:00:00.242000+00:00
O-20200103-100004-001-000-2,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-002,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-002,MARKET,SELL,1000000,...,None,None,None,None,None,None,None,f618bc7b-53ac-4933-9023-ea3b9683953a,2020-01-03 10:00:04.136000+00:00,2020-01-03 10:00:04.136000+00:00
O-20200103-100006-001-000-3,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-003,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-003,MARKET,SELL,1000000,...,None,None,None,None,None,None,None,60e4c0f6-b5f8-4bf7-9fd2-4b659f2d1eff,2020-01-03 10:00:06.093000+00:00,2020-01-03 10:00:06.093000+00:00
O-20200103-100007-001-000-4,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-004,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-004,MARKET,BUY,1000000,...,None,None,None,None,None,None,None,d0a05e7b-7352-4bbf-9906-a597ab968fe9,2020-01-03 10:00:07.548000+00:00,2020-01-03 10:00:07.548000+00:00
O-20200103-165230-001-000-5,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-005,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-005,MARKET,SELL,1000000,...,None,None,None,None,None,None,None,00c024b9-bfa9-4956-9d4a-d6dc9805d758,2020-01-03 16:52:30.901000+00:00,2020-01-03 16:52:30.901000+00:00
O-20200103-165246-001-000-6,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-006,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-006,MARKET,BUY,1000000,...,None,None,None,None,None,None,None,c98b5c3a-fc4a-4bc8-a9d0-93876a61a293,2020-01-03 16:52:46.001000+00:00,2020-01-03 16:52:46.001000+00:00
O-20200105-170032-001-000-7,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-007,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-007,MARKET,BUY,1000000,...,None,None,None,None,None,None,None,c40ec2be-e2ec-4ad7-be21-29515324b5bf,2020-01-05 17:00:32.758000+00:00,2020-01-05 17:00:32.758000+00:00
O-20200105-170530-001-000-8,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-008,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-008,MARKET,SELL,1000000,...,None,None,None,None,None,None,None,7dbe1b7f-7c5a-4b3b-87d0-197bac03d38c,2020-01-05 17:05:30.628000+00:00,2020-01-05 17:05:30.628000+00:00
O-20200108-031312-001-000-9,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-009,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-009,MARKET,SELL,1000000,...,None,None,None,None,None,None,None,363b55d7-fe8b-4f0a-9733-016dbfa23f52,2020-01-08 03:13:12.834000+00:00,2020-01-08 03:13:12.834000+00:00


In [14]:
engine.trader.generate_positions_report()

,trader_id,strategy_id,instrument_id,account_id,opening_order_id,closing_order_id,entry,side,quantity,peak_qty,ts_init,ts_opened,ts_last,ts_closed,duration_ns,avg_px_open,avg_px_close,commissions,realized_return,realized_pnl
position_id,,,,,,,,,,,,,,,,,,,,
EUR/USD.SIM-MACDStrategy-000-568e282c-f908-4f64-bdd2-9d5aa3c59932,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-001,O-20200103-100000-001-000-1,O-20200103-100004-001-000-2,BUY,FLAT,0,1000000,1578045600242000000,2020-01-03 10:00:00.242000+00:00,1578045604136000000,2020-01-03 10:00:04.136000+00:00,3894000000,1.11644,1.11635,[44.66 USD],-0.00008,-134.66 USD
EUR/USD.SIM-MACDStrategy-000-84a7490b-2ce8-4b68-aa1b-b340706ae6cf,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-001,O-20200103-100006-001-000-3,O-20200103-100007-001-000-4,SELL,FLAT,0,1000000,1578045606093000000,2020-01-03 10:00:06.093000+00:00,1578045607548000000,2020-01-03 10:00:07.548000+00:00,1455000000,1.11604,1.11637,[44.65 USD],-0.00030,-374.65 USD
EUR/USD.SIM-MACDStrategy-000-6bdd6948-65c6-4ea9-bef4-703a68fcb2ef,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-001,O-20200103-165230-001-000-5,O-20200103-165246-001-000-6,SELL,FLAT,0,1000000,1578070350901000000,2020-01-03 16:52:30.901000+00:00,1578070366001000000,2020-01-03 16:52:46.001000+00:00,15100000000,1.11534,1.11543,[44.62 USD],-0.00008,-134.62 USD
EUR/USD.SIM-MACDStrategy-000-f4bff5de-349d-49f6-bc53-b112ffb14b8b,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-001,O-20200105-170032-001-000-7,O-20200105-170530-001-000-8,BUY,FLAT,0,1000000,1578243632758000000,2020-01-05 17:00:32.758000+00:00,1578243930628000000,2020-01-05 17:05:30.628000+00:00,297870000000,1.11703,1.11652,[44.67 USD],-0.00046,-554.67 USD
EUR/USD.SIM-MACDStrategy-000-d20fa694-1e3d-4ef5-8150-1b196aead6ff,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-001,O-20200108-031312-001-000-9,O-20200108-031316-001-000-10,SELL,FLAT,0,1000000,1578453192834000000,2020-01-08 03:13:12.834000+00:00,1578453196947000000,2020-01-08 03:13:16.947000+00:00,4113000000,1.11403,1.11360,[44.55 USD],0.00039,385.45 USD
EUR/USD.SIM-MACDStrategy-000-15c9149d-8376-4fd0-8dc8-d25b140bf66c,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-001,O-20200108-062139-001-000-11,O-20200108-062157-001-000-12,SELL,FLAT,0,1000000,1578464499588000000,2020-01-08 06:21:39.588000+00:00,1578464517525000000,2020-01-08 06:21:57.525000+00:00,17937000000,1.11231,1.11211,[44.49 USD],0.00018,155.51 USD
EUR/USD.SIM-MACDStrategy-000-5f03d540-8908-48a1-a25f-f66deacaa55d,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-001,O-20200108-081501-001-000-13,O-20200108-081514-001-000-14,SELL,FLAT,0,1000000,1578471301502000000,2020-01-08 08:15:01.502000+00:00,1578471314869000000,2020-01-08 08:15:14.869000+00:00,13367000000,1.11121,1.11120,[44.44 USD],0.00001,-34.44 USD
EUR/USD.SIM-MACDStrategy-000,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-001,O-20200108-172644-001-000-15,O-20200108-172650-001-000-16,BUY,FLAT,0,1000000,1578504404662000000,2020-01-08 17:26:44.662000+00:00,1578504410316000000,2020-01-08 17:26:50.316000+00:00,5654000000,1.11164,1.11121,[44.45 USD],-0.00039,-474.45 USD


In [15]:
engine.trader.generate_account_report(Venue("SIM"))

,total,locked,free,currency,account_id,account_type,base_currency,margins,reported,info
2020-01-01 17:00:10.447000+00:00,1000000.00,0.00,1000000.00,USD,SIM-001,MARGIN,USD,[],True,{}
2020-01-03 10:00:00.242000+00:00,999977.67,33493.20,966484.47,USD,SIM-001,MARGIN,USD,"[{'type': 'MarginBalance', 'initial': '0.00', ...",False,{}
2020-01-03 10:00:04.136000+00:00,999865.34,0.00,999865.34,USD,SIM-001,MARGIN,USD,[],False,{}
2020-01-03 10:00:06.093000+00:00,999843.02,33481.20,966361.82,USD,SIM-001,MARGIN,USD,"[{'type': 'MarginBalance', 'initial': '0.00', ...",False,{}
2020-01-03 10:00:07.548000+00:00,999490.69,0.00,999490.69,USD,SIM-001,MARGIN,USD,[],False,{}
2020-01-03 16:52:30.901000+00:00,999468.38,33460.20,966008.18,USD,SIM-001,MARGIN,USD,"[{'type': 'MarginBalance', 'initial': '0.00', ...",False,{}
2020-01-03 16:52:46.001000+00:00,999356.07,0.00,999356.07,USD,SIM-001,MARGIN,USD,[],False,{}
2020-01-05 17:00:32.758000+00:00,999333.73,33510.90,965822.83,USD,SIM-001,MARGIN,USD,"[{'type': 'MarginBalance', 'initial': '0.00', ...",False,{}
2020-01-05 17:05:30.628000+00:00,998801.40,0.00,998801.40,USD,SIM-001,MARGIN,USD,[],False,{}
2020-01-08 03:13:12.834000+00:00,998779.12,33420.90,965358.22,USD,SIM-001,MARGIN,USD,"[{'type': 'MarginBalance', 'initial': '0.00', ...",False,{}
